<a href="https://colab.research.google.com/github/augustolnb/electrical_engineering/blob/main/malha_aterramento-v01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

#### Cálculo do condutor

In [2]:
# Dimensionamento dos condutores

# t_amb -> temperatura ambiente
# t_max_malha -> temperatura máxima do cabo da malha
# t_max_lig -> temperatura máxima do cabo de ligação
# s_cond -> seção transversal do condutor [mm²]
# i_falta -> corrente de falta
# t_falta -> tempo de falta
#
#
#

t_falta = 0.5
t_amb = 30
t_max_malha = 450
t_max_lig = 450
i_falta = 3200
i_malha = 1220

# Calculo do condutor da malha
#i_falta = float(input("Corrente máxima de falta [A]: "))
# Corrente de malha = 60% corrente de falta

# calculo da corrente de falta, totalmente diferente do que consta no pdf
#i_falta = 226.53 * s_cond_malha * np.sqrt((1/t_falta) * np.log((t_max_malha - t_amb) / (234 + t_amb) + 1))
#i_malha = 0.6 * i_falta

s_cond_lig = i_falta / (226.53 * np.sqrt((1/t_falta) * np.log((t_max_malha - t_amb) / (234 + t_amb) + 1)))
s_cond_malha = i_malha / (226.53 * np.sqrt((1/t_falta) * np.log((t_max_malha - t_amb) / (234 + t_amb) + 1)))

# Adicionar condicional para setar valores padronizados de condutores
'''
if(s_cond_lig < 35):
    s_cond_lig = 35
    print("Valor mínimo prático é de 35mm²")
if(s_cond_malha < 35):
    s_cond_malha = 35
    print("Valor mínimo prático é de 35mm²")
'''

# Diâmetro do condutor em metros [d >= 0.25.h]
diametro_cond = (np.sqrt(4 * s_cond_lig / np.pi)) / 1000


print("Condutor de ligação: {} mm²".format(np.round(s_cond_lig, 2)))
print("\nCondutor de malha: {} mm²".format(np.round(s_cond_malha, 2)))
print("\nDiâmetro do condutor de ligação: {} m".format(np.round(diametro_cond, 5)))


Condutor de ligação: 10.24 mm²

Condutor de malha: 3.9 mm²

Diâmetro do condutor de ligação: 0.00361 m


#### Cálculo da Resistência da Malha

In [2]:
# Resistência de uma haste isolada
#
#   R = (rho_aparente/(np.pi*2*L)*np.ln(4*L/d))
#   
#   onde:
#   L               -> Comprimento da haste [m]
#   d               -> Diâmetro da haste [m]
#   rho_aparente    -> Resistividade aparente do solo [Ohm.m]
#   h               -> Profundidade da malha [m]
#   C_malha         -> Comprimento da malha [m]
#   L_malha         -> Largura da malha [m]
#   e_a             -> Espaçamento dos condutores no comprimento
#   e_b             -> Espaçamento dos condutores na largura
#   n_hastes        -> Numº de hastes [un]
#   comp_total_cond -> Comprimento total do condutores [m]

C_malha = 5
L_malha = 10
h = 0.6
L = 5
d = 0.025
rho_aparente = 411.8
e_a = 5 # Espaçamentos entre os condutores da malha, no comprimento
e_b = 5 # Espaçamentos entre os condutores da malha, na largura
e = 5 # Espaçamentos entre as hastes

area_total = C_malha * L_malha
n_a = int((C_malha/e_a) + 1)
n_b = int((L_malha/e_b) + 1)
comp_total_cond = C_malha * n_b + L_malha * n_a
#comp_total_hastes = n_hastes * L
#comp_total = comp_total_cond + comp_total_hastes

#print(comp_total)

#######################################################
# Calcular Resistência

# Resistência de uma haste individual - Diagonal principal da matriz de resistências
R = (rho_aparente/(np.pi*2*L)) * np.log(4*L/d)
print("Resistencia de uma haste individual: {} Ohms\n".format(np.round(R, 2)))

Resistencia de uma haste individual: 87.62 Ohms



In [3]:
# Gera vetor de hastes, o numº de cada haste é a chave para um 
# Dicionário que contém as coordenadas daquela haste

hastes = []
for i in range(n_b):
    hastes.append(np.round(np.arange(0, C_malha+1 , e), 1))

n_hastes = len(hastes) * len(np.arange(0, C_malha+1 , e))
hastes = np.array(hastes).reshape((1, n_hastes))

for i in range(n_hastes):
    hastes[0,i] = i+1
print(hastes)

[[1 2 3 4 5 6]]


In [5]:
cont = 1
coordenadas = dict()
k = 0
while k < n_b:
    for j in range(n_a):
#        print("\n\nCoordenadas: ")
#        print("({},{})".format(j*e,k*e))
        coordenadas[cont] = j*e, k*e
        cont += 1
    k += 1

In [27]:
haste_ref = coordenadas[3]
print(haste_ref)
hastezinha = coordenadas[5]
print(hastezinha)

x = np.sqrt((haste_ref[0] - hastezinha[0])**2 + (haste_ref[1] - hastezinha[1])**2)
b = np.sqrt(x**2 + L**2)

r_int = (411.8 / (4*np.pi*L)) * (np.log(((b+L)**2 - x**2) / (x**2 - (b-L)**2)))

print(np.round(r_int,2))


(0, 5)
(0, 10)
11.55


In [33]:
# Cálculo da matriz de resistências 
# Matriz triangular, da diagonal para baixo ela repete o padrão da parte superior
r = np.zeros((n_hastes,n_hastes), dtype=float)

for i in range(n_hastes): # laço para percorrer linhas da matriz de coordenadas
    haste_ref = coordenadas[i]
    for j in range(n_hastes): # laço para percorrer colunas da matriz de coordenadas
        if(i==j): 
            r[i,j] = R
            continue
        elif(i < j):
            x = np.sqrt((coord_h[i,i] - coord_h[])**2 + ()**2) # distancia entre as hastes



(35, 35)


In [ ]:
for i in range(r.shape[0]):
    for j in range(r.shape[1]):
#        print(r[i,j])
        if(i == j):
            continue
#        x = np.sqrt((r[i,i] - r[i,j])**2 + (i*e_b - )**2)

#print(r[])

In [41]:
b = float(input("Valor da diagonal: "))
e = float(input("Espaçamento superficial entre as hastes: "))
r_ind = (411.8 / (4*np.pi*L)) * (np.log(((b+L)**2 - e**2) / (e**2 - (b-L)**2)))
print(np.round(r_ind,2))

Valor da diagonal: 12.24
Espaçamento superficial entre as hastes: 11.2
5.61
